In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split

In [23]:
def loadData(filePath):
    data = pd.read_csv(filePath)
    threshold = 64
    if 'label' in data.columns:
        X = np.array(data.iloc[:,1:])
        # 8位 256 -> 1位 2 减小复杂度
        X[X<=threshold], X[X>threshold] = 0, 1
        y = np.array(data.iloc[:,0])
        return X, y
    else:
        X = np.array(data.iloc[:,:])
        # 8位 256 -> 1位 2 减小复杂度
        X[X<=threshold], X[X>threshold] = 0, 1
        return X

In [95]:
class TreeNode:
    def __init__(self):
        self.featureID = None
        self.featureValue = None
        self.trueBranch = None
        self.falseBranch = None
        self.klass = None


class BoostingTree:
    def __init__(self):
        self.maxDepth = 100
        self.eps = 1e-4
        
    def differentValues(self, arr):
        return len(set(arr))
    
    def maxCount(self, arr, weight = None):
        if weight is None:
            weight = np.ones(arr.shape[0])
        s = set(arr)
        dic = dict()
        for key in s:
            dic[key] = 0.
        for i in range(arr.shape[0]):
            dic[arr[i]] += weight[i]
        val = arr[0]
        for key in dic.keys():
            if dic[key] > dic[val]:
                val = key
        return val
    
    # 找最佳切分点
    def findBestSplit(self, X, y, weight):
        featureID, featureValue, err = -1, -1, 1
        trueList, falseList = None, None
        for i in range(X.shape[1]):
            featureValues = set(X[:,i])
            if len(featureValues) == 1:
                continue
            for val in featureValues:
                Tlist, Flist = np.arange(X.shape[0])[X[:,i]==val], np.arange(X.shape[0])[X[:,i]!=val]
                Ttag, Ftag = self.maxCount(y[Tlist], weight[Tlist]), self.maxCount(y[Flist], weight[Flist])
                curERR = weight[Tlist][y[Tlist]!=Ttag].sum() + weight[Flist][y[Flist]!=Ftag].sum()
                if curERR < err:
                    err = curERR
                    featureID, featureValue = i, val
                    trueList, falseList = Tlist, Flist
#         print('featureID = %d, featureValue = %d, err = %.8f.' % (featureID, featureValue, err))
        return featureID, featureValue, trueList, falseList
    
    # 根据weight建树
    def buildTree(self, X, y, weight, depth):
#         print('depth = %d, X shape = (%d,%d).' % (depth, X.shape[0], X.shape[1]))
        curNode = TreeNode()
        if self.differentValues(y) == 1 or depth >= self.maxDepth or X.shape[0] < self.minLeafNodes:
            curNode.klass = self.maxCount(y, weight)
            return curNode
        featureID, featureValue, trueList, falseList = self.findBestSplit(X, y, weight)
        if featureID == -1:
            curNode.klass = self.maxCount(y, weight)
            return curNode
        curNode.featureID = featureID
        curNode.featureValue = featureValue
        curNode.trueBranch = self.buildTree(X[trueList,:], y[trueList], weight[trueList], depth + 1)
        curNode.falseBranch = self.buildTree(X[falseList, :], y[falseList], weight[falseList], depth + 1)
        return curNode
    
    def __predict(self, root, x):
        if root.klass != None:
            return root.klass
        if x[root.featureID] == root.featureValue:
            return self.__predict(root.trueBranch, x)
        else:
            return self.__predict(root.falseBranch, x)
    def sgn(self, x):
        return 1 if x >= 0 else -1
    
    def fit(self, X, y, maxDepth = 100, treeNum = 100, minLeafNodes = 10, eps = 1e-4):
        self.maxDepth = maxDepth  # 最大深度
        self.treeNum = treeNum    # 树的最多数量
        self.minLeafNodes = minLeafNodes    # 叶子节点最少样本数
        self.eps = eps            # 误差小于eps停止建树
        self.N, self.m = X.shape
        self.G = []
        self.alpha = []
        self.f = np.zeros(self.N)
        self.weight = np.array([1/self.N for _ in range(self.N)])
        for m in range(self.treeNum):
            self.G.append(self.buildTree(X, y, self.weight, 0))
            Gm = np.zeros(self.N)
            err = 0
            for i in range(self.N):
                Gm[i] = self.__predict(self.G[-1], X[i])
                if Gm[i] != y[i]:
                    err += self.weight[i]
            alpha = (0.5 * np.log((1 - err) / err))
            self.alpha.append(alpha)
            Zm = 0
            for i in range(self.N):
                Zm += self.weight[i] * np.exp(-alpha * y[i] * Gm[i])
            for i in range(self.N):
                self.weight[i] = self.weight[i] / Zm * np.exp(-alpha * y[i] * Gm[i])
            totalErr = 0
            for i in range(self.N):
                self.f[i] += alpha * Gm[i]
                if self.sgn(self.f[i]) != y[i]:
                    totalErr += 1
            totalErr = totalErr / self.N
            print('step = %d, error = %f.' % (m + 1, totalErr))
    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            for j in range(len(self.G)):
                y_pred[i] += self.alpha[j] * self.__predict(self.G[j], X[i])
        return y_pred

In [97]:
trainFilePath = '../mnist/train.csv'
X, y = loadData(trainFilePath)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.4, random_state = 42, stratify = y)

In [99]:
models = [BoostingTree() for _ in range(10)]
bin_y_train = [np.array(list(map(lambda x : 1 if x == i else -1,y_train))) for i in range(10)]
start = time.time()
for i in range(10):
    models[i].fit(X_train, bin_y_train[i], maxDepth = 1, treeNum = 20, minLeafNodes = 10)
    end = time.time()
    print('Model %d has been trained. Cost %fs.' % (i, end - start))
    start = end
    

step = 1, error = 0.077976.
step = 2, error = 0.077976.
step = 3, error = 0.063611.
step = 4, error = 0.067421.
step = 5, error = 0.051032.
step = 6, error = 0.037897.
step = 7, error = 0.036786.
step = 8, error = 0.032937.
step = 9, error = 0.034127.
step = 10, error = 0.034167.
step = 11, error = 0.032738.
step = 12, error = 0.025397.
step = 13, error = 0.031944.
step = 14, error = 0.025556.
step = 15, error = 0.026984.
step = 16, error = 0.023889.
step = 17, error = 0.023730.
step = 18, error = 0.022381.
step = 19, error = 0.021944.
step = 20, error = 0.020754.
Model 0 has been trained. Cost 272.215871s.
step = 1, error = 0.111508.
step = 2, error = 0.111508.
step = 3, error = 0.128333.
step = 4, error = 0.111468.
step = 5, error = 0.064603.
step = 6, error = 0.045992.
step = 7, error = 0.045952.
step = 8, error = 0.038016.
step = 9, error = 0.046190.
step = 10, error = 0.038016.
step = 11, error = 0.038690.
step = 12, error = 0.033849.
step = 13, error = 0.033810.
step = 14, error 

In [100]:
y_prob = []
for i in range(10):
    y_prob.append(models[i].predict(X_valid))
y_prob = np.array(y_prob)
y_pred = np.argmax(y_prob, axis = 0)
acc = (y_pred==y_valid).sum() / y_pred.shape[0]
print('acc = %f.' % (acc))

acc = 0.816369.


In [101]:
testFilePath = '../mnist/test.csv'
X_test = loadData(testFilePath)
y_prob = []
for i in range(10):
    y_prob.append(models[i].predict(X_test))
y_prob = np.array(y_prob)
y_pred = y_prob.argmax(axis = 0)
ans = pd.DataFrame({'ImageId' : np.arange(1, y_pred.shape[0] + 1), 'Label' : y_pred.astype(np.int32)})
ans.to_csv('./result_mnist.csv', index = False)
# maxDepth = 1, treeNum = 20, minLeafNodes = 10, kaggle准确率 81.264% 深度1只有树桩

-1